#### Merging intervals


About the pattern
The merge intervals pattern deals with problems involving overlapping intervals. Each interval is represented by a start and an end time. For example, an interval of [10,20] start at 10 and end at 20. This pattern involves tasks such as merging intersecting intervals, inserting new intervals into existing sets, or determining the minimum number of intervals needed to cover a given range. The most common problems solved using this pattern are event scheduling, resource allocation, and time slot consolidation.

The key to understanding this pattern and exploiting its power lies in understanding how any two intervals may overlap. The illustration below shows different ways in which two intervals can relate to each other:

e.g. 
* [1,4], [3,7] -> [1,7]
* [1,4], [3,7], [9,12] -> [1,7], [9,12]

Application:
1. Merge intervals: Given a sorted list of intervals, merge all overlapping intervals.
2. Meeting rooms: Given an array of meeting time intervals consisting of start and end times, determine if a person could attend all meetings.


#### Q1
We are given an array of closed intervals, intervals, where each interval has a start time and an end time. The input array is sorted with respect to the start times of each interval. For example, intervals = 
[ [1,4], [3,6], [7,9] ]
 is sorted in terms of start times 1, 3, and 7.

Your task is to merge the overlapping intervals and return a new output array consisting of only the non-overlapping intervals.

e.g. [[1,5],[3,7],[4,6],[6,8]] are overlapping -> merging [1,8]



In this solution, we use the merge intervals pattern with a simple linear scan to merge the overlapping intervals. First, we create an output list and copy the first interval of the input list to it. Next, we traverse the remaining intervals of the input list and check whether any interval overlaps with the interval present in the output list. If they overlap, update the interval in the output list. Otherwise, add the current input interval to the output list. Repeat this for all the intervals in the input list. Please note that when we have more than one interval in the output list, we compare the input intervals with the last interval of the output list.

* Inside the loop, we check each interval of the input list against the last interval of the output list. For each interval in the input list, we do the following:

* If the current input interval is overlapping with the last interval in the output list, we merge these two intervals and replace the last interval of the output list with the newly merged interval.
Otherwise, we add the input interval to the output list.

To check if the current input interval and the last interval in the output list overlap, we’ll check the start time of the current interval and the end time of the last interval in the output list. If the start time of the current interval is less than the end time of the last interval in the output list, that is, curr_start <=> prev_end, the two intervals overlap. Otherwise, they don’t overlap. Since the intervals are sorted in terms of their start times, we won’t encounter cases where the current interval’s start and end times are less than the start time of the last interval in the output list.

time complexity: O(n)
space complexity: O(1)

In [ ]:
# optimized solution
def merge_intervals(intervals):
    interval_merge = []

    for element in intervals:
        
        if not interval_merge:
            interval_merge.append(element)
            continue
        
        element_small = element[0] # element_small > interval
        element_large = element[1]

        interval_merge_len = len(interval_merge)
        
        # traverse interval merge list
        idx_insert = 0

        # comparing interval_merge from right to left
        # only need to compare last element
        last_min = interval_merge[interval_merge_len-1][0]
        last_max = interval_merge[interval_merge_len-1][1]
        if element_small > last_max:
            interval_merge.append(element)
        elif element_large > last_max:
            interval_merge[interval_merge_len-1][1] = element_large
        else:
            continue
    return interval_merge

#### Question 2

Given a sorted list of nonoverlapping intervals and a new interval, your task is to insert the new interval into the correct position while ensuring that the resulting list of intervals remains sorted and nonoverlapping. Each interval is a pair of nonnegative numbers, the first being the start time and the second being the end time of the interval.



Below is my complex solution: Although it's optmized, the coding is very complex.
* you need to be very careful about return. you should only do return in the last unless obvious except happens in the middle of loop.
* add more space complexity to store output list instead of delete list elements directly
* you can add boolean condition


In [3]:
def insert_interval(existing_intervals, new_interval):
  
  if not existing_intervals:
    return [new_interval]
  
  if not new_interval:
    return existing_intervals


  # for a given new interval, traverse existing interval
  # merge with existing interval
  # then take the new current idx interval to merge with future interval
  n = len(existing_intervals)
  low_val = new_interval[0]
  high_val = new_interval[1]

  for i in range(n):
    
    current_left = existing_intervals[i][0]
    current_right = existing_intervals[i][1]


    # check if merge or not
    ####### not merged condition: #########
    if low_val > current_right:
      # unless it's last idx, continue traversing
      if i==n-1:
        existing_intervals.append(new_interval)
        return existing_intervals
      continue
    
    if high_val < current_left:
      # insert existing interval 
      existing_intervals.insert(i,new_interval)
      return existing_intervals
    
    ########## merging conditions ###########:
    # (1) completely merging
    if high_val <= current_right and low_val >= current_left:
      return existing_intervals
    
    # (2) modified left boundary
    if low_val < current_left:
      # replace current left with low_val
      existing_intervals[i][0] = low_val
    
    # (3) if right boundary needs to be modified, perform dynamical comparison
    if high_val > current_right:
      existing_intervals[i][1] = high_val
      # if last 1, return modified list
      if i == n-1:
        return existing_intervals
      
      # check next intervals whether need to be merged or not
      nxt_high = existing_intervals[i+1][1]
      nxt_low = existing_intervals[i+1][0]

      while high_val > nxt_high:
        
        # delete next interval list due to merging
        del existing_intervals[i+1]
        if i+1 == n-1:
          existing_intervals[i][1] = nxt_high
          return existing_intervals
        # update
        nxt_high = existing_intervals[i+1][1]
        nxt_low = existing_intervals[i+1][0]

      
      if high_val >= nxt_low:
        existing_intervals[i][1] = nxt_high
        del existing_intervals[i+1]
      
      
    return existing_intervals

A better one:

In [7]:
def insert_interval(existing_intervals, new_interval):
    new_start, new_end = new_interval[0], new_interval[1]
    i = 0
    n = len(existing_intervals)
    output = []

    # add all intervals that come before new_interval
    # if there is overlap on the right, modify the right boundary
    while i < n and existing_intervals[i][0] < new_start:
        output.append(existing_intervals[i])
        i = i + 1
    
    # add new interval if there is no overlap
    # or merge overlapping intervals
    if not output or output[-1][1] < new_start:
        output.append(new_interval)
    else:
        output[-1][1] = max(output[-1][1], new_end)
    
    # add all intervals that come after new_interval
    while i < n:
        ei = existing_intervals[i]
        start, end = ei[0], ei[1]
        if output[-1][1] < start:
            output.append(ei)
        else:
            # since it's sorted, you only need to compare the right boundary
            output[-1][1] = max(output[-1][1], end)
        i += 1
    return output

# Driver code
def main():
    new_interval = [[5, 7], [8, 9], [10, 12], [1, 3], [1, 10]]
    existing_intervals = [
        [[1, 2], [3, 5], [6, 8]],
        [[1, 3], [5, 7], [10, 12]],
        [[8, 10], [12, 15]],
        [[5, 7], [8, 9]],
        [[3, 5]]
    ]
    
    for i in range(len(new_interval)):
        print(i + 1, ".\tExiting intervals: ", existing_intervals[i], sep="")
        print("\tNew interval: ", new_interval[i], sep="")
        output = insert_interval(existing_intervals[i], new_interval[i])
        print("\tUpdated intervals: ", output, sep = "")
        print("-"*100)


if __name__ == "__main__":
    main()

1.	Exiting intervals: [[1, 2], [3, 5], [6, 8]]
	New interval: [5, 7]
	Updated intervals: [[1, 2], [3, 8]]
----------------------------------------------------------------------------------------------------
2.	Exiting intervals: [[1, 3], [5, 7], [10, 12]]
	New interval: [8, 9]
	Updated intervals: [[1, 3], [5, 7], [8, 9], [10, 12]]
----------------------------------------------------------------------------------------------------
3.	Exiting intervals: [[8, 10], [12, 15]]
	New interval: [10, 12]
	Updated intervals: [[8, 15]]
----------------------------------------------------------------------------------------------------
4.	Exiting intervals: [[5, 7], [8, 9]]
	New interval: [1, 3]
	Updated intervals: [[1, 3], [5, 7], [8, 9]]
----------------------------------------------------------------------------------------------------
5.	Exiting intervals: [[3, 5]]
	New interval: [1, 10]
	Updated intervals: [[1, 10]]
------------------------------------------------------------------------------